## ETL - Extract, Transform and Load

En esta etapa del proyecto, se llevara a cabo el proceso de ETL (extracción, transformación y carga), para poder comenzar con la obtención de valiosa e importante información a partir de los datos. En esta etapa podremos dimensionar la información con la que contamos, las diversas estructuras y formatos de los datos, las relaciones entre los distintos datasets que trabajaremos, para su porterior utilización, y que esta, sea correcta, efectiva y eficaz.

Comenzamos con la extracción de los datos de los tres datasets disponibles:
- output_steam_games.json
- australian_user_reviews.json
- australian_user_items.json

**A continuación, importamos las librerias necesarias para analizar nuestros datasets:**

In [1]:
import pandas as pd
import json
import ast
import FuncionExtra as f
import pyarrow as pa
import pyarrow.parquet as pq

### Extracción de los datos del dataset 'output_steam_games.json':


In [7]:
#Leemos el archivo 'output_steam_games.json', de tipo json, con pandas y lo guardamos en una variable como un DataFrame.

games = pd.read_json('../output_steam_games.json', lines= True)

df_games = pd.DataFrame(games)

### Extracción de los datos del dataset 'australian_user_reviews.json':

In [8]:
#Para leer el archivo 'australian_user_reviews.json', usamos esta función y no lo hacemos como hicimos con el archivo anterior, debido a erores de 
#sintaxis. Se crea una lista vacia donde se ira almacenando linea por linea. Se utiliza 'with' para abrir el archivo y cerrarlo correctamente 
#posteriormente. Con un bucle for, se recorre linea por linea, y se almacena en la lista creada al inicio. Por ultimo, transformamos el objeto de
#Python a un DataFrame.

reviews = []

with open('../australian_user_reviews.json', encoding= 'utf-8') as file:
    for line in file.readlines():
        reviews.append(ast.literal_eval(line))

df_reviews = pd.DataFrame(reviews)

### Extracción de los datos del dataset 'australian_users_items.json':

In [9]:
#Para la extracción de datos, del archivo 'australian_users_items.json', se utilizo la misma función que antes.

items = []

with open('../australian_users_items.json', encoding= 'utf-8') as file:
    for line in file.readlines():
        items.append(ast.literal_eval(line))

df_items = pd.DataFrame(items)

### Guardado de los DataFrames en csv, para abrirlos de forma más rapida

Guardado de DataFrame df_games como csv:

In [10]:
df_games.to_csv('../steam_games.csv', encoding= 'utf-8', index= False)

Guardado de DataFrame df_reviews como csv:

In [11]:
df_reviews.to_csv('../user_reviews.csv', encoding= 'utf-8', index= False)

Guardado de DataFrame df_items como csv:

In [12]:
df_items.to_csv('../user_items.csv', encoding= 'utf-8', index= False)

### Lectura de los csv guardados anteriormente:

In [13]:
steam_games = pd.read_csv(r'../steam_games.csv', low_memory= False)

In [14]:
user_reviews = pd.read_csv(r'../user_reviews.csv')

In [15]:
user_items = pd.read_csv(r'../user_items.csv')

### Transformación de los datos:

In [16]:
#Analizamos el tamaño de los DataFrames, cantidad de filas y columnas de cada uno.

datasets= {'steam_games': steam_games, 'user_reviews': user_reviews, 'user_items': user_items}

f.tamanio_datsets(datasets)

El tamaño del DataFrame steam_games es: (120445, 13)
El tamaño del DataFrame user_reviews es: (25799, 3)
El tamaño del DataFrame user_items es: (88310, 5)


#### Información general, valores nulos y porcentaje de valores faltantes del DataFrame *df_games*

In [17]:
#Información general:

steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24064 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [18]:
#Analisis de valores nulos.

steam_games.isna().sum()

publisher       96381
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [19]:
#Analisis de porcentaje de valores nulos por columna.

f.porcentaje_valores_nulos(steam_games)

La columna publisher tiene un  80.02 % de valores nulos
La columna genres tiene un  76.05 % de valores nulos
La columna app_name tiene un  73.32 % de valores nulos
La columna title tiene un  75.02 % de valores nulos
La columna url tiene un  73.32 % de valores nulos
La columna release_date tiene un  75.04 % de valores nulos
La columna tags tiene un  73.46 % de valores nulos
La columna reviews_url tiene un  73.32 % de valores nulos
La columna specs tiene un  73.88 % de valores nulos
La columna price tiene un  74.46 % de valores nulos
La columna early_access tiene un  73.32 % de valores nulos
La columna id tiene un  73.32 % de valores nulos
La columna developer tiene un  76.06 % de valores nulos


In [20]:
steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Información general del DataFrame *df_reviews*

No posee datos faltantes, por lo que no analisamos la cantidad ni el porcentaje de valores nulos.

In [21]:
#Información general

user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [22]:
user_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


#### Información general del DataFrame *df_items*

No posee datos faltantes, por lo que no analisamos la cantidad ni el porcentaje de valores nulos.

In [23]:
#Información general:

user_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  int64 
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.4+ MB


In [24]:
user_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


### Desanidamos registros anidados:

#### Detectamos las columnas anidadas en cada DataFrame:

In [25]:
for nombre, dataframe in datasets.items():
    print(f'Las columnas anidadas en el DataFrame {nombre}, son {f.deteccion_columnas_anidadas(dataframe)}')

Las columnas anidadas en el DataFrame steam_games, son []
Las columnas anidadas en el DataFrame user_reviews, son ['reviews']
Las columnas anidadas en el DataFrame user_items, son ['items']


#### Visualizamos los registros que tienen las columnas con registros anidados:

In [26]:
user_reviews['reviews'][0]

'[{\'funny\': \'\', \'posted\': \'Posted November 5, 2011.\', \'last_edited\': \'\', \'item_id\': \'1250\', \'helpful\': \'No ratings yet\', \'recommend\': True, \'review\': \'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.\'}, {\'funny\': \'\', \'posted\': \'Posted July 15, 2011.\', \'last_edited\': \'\', \'item_id\': \'22200\', \'helpful\': \'No ratings yet\', \'recommend\': True, \'review\': "It\'s unique and worth a playthrough."}, {\'funny\': \'\', \'posted\': \'Posted April 21, 2011.\', \'last_edited\': \'\', \'item_id\': \'43110\', \'helpful\': \'No ratings yet\', \'recommend\': True, \'review\': \'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!\'}

In [27]:
user_items['items'][0]

'[{\'item_id\': \'10\', \'item_name\': \'Counter-Strike\', \'playtime_forever\': 6, \'playtime_2weeks\': 0}, {\'item_id\': \'20\', \'item_name\': \'Team Fortress Classic\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'30\', \'item_name\': \'Day of Defeat\', \'playtime_forever\': 7, \'playtime_2weeks\': 0}, {\'item_id\': \'40\', \'item_name\': \'Deathmatch Classic\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'50\', \'item_name\': \'Half-Life: Opposing Force\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'60\', \'item_name\': \'Ricochet\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'70\', \'item_name\': \'Half-Life\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'130\', \'item_name\': \'Half-Life: Blue Shift\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'300\', \'item_name\': \'Day of Defeat: Source\', \'playtime_forever\': 4733, \'playtime_2weeks\': 0}, {\'ite

### Desanidamos las columnas anidadas

Para desanidar, ambas columnas, de ambos DataFrames, utilizamos la lista obtenida al comienzo, cuando abrimos el archivo json.

#### Desanidamos la columna 'reviews' de DataFrame 'user_reviews'

In [28]:
reviews_desanidado = pd.json_normalize(reviews, record_path = ['reviews'], meta = ['user_id', 'user_url'])

In [29]:
reviews_desanidado

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637
...,...,...,...,...,...,...,...,...,...
59300,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
59301,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
59302,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,LydiaMorley,http://steamcommunity.com/id/LydiaMorley
59303,,Posted July 20.,,730,No ratings yet,True,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley


#### Desanidamos la columna 'items' de DataFrame 'user_items'

In [30]:
items_desanidado = pd.json_normalize(items, record_path = ['items'], meta = ['user_id','items_count','steam_id', 'user_url'])

In [31]:
items_desanidado

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


### Vemos el tamaño de los DataFrames desanidados

In [32]:
dataframes_desanidados = {'steam_games': steam_games, 'reviews_desanidado': reviews_desanidado, 'items_desanidado': items_desanidado}

f.tamanio_datsets(dataframes_desanidados)

El tamaño del DataFrame steam_games es: (120445, 13)
El tamaño del DataFrame reviews_desanidado es: (59305, 9)
El tamaño del DataFrame items_desanidado es: (5153209, 8)


Notamos que el tamaño de los DataFrames 'user_reviews' y 'user_items', aumento notablemente después de desanidar sus columnas anidadas. Este crecimiento se ve tanto en la cantidad de filas, como en la de columnas.

### Analizamos filas duplicadas y valores nulos, después de desanidar:

#### Filas duplicadas y valores nulos en 'steam_games':

In [33]:
#Vemos los valores nulos por columna:

steam_games.isnull().sum()

publisher       96381
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [34]:
#Vemos otra vez, el porcentaje de valores nulos:

f.porcentaje_valores_nulos(steam_games)

La columna publisher tiene un  80.02 % de valores nulos
La columna genres tiene un  76.05 % de valores nulos
La columna app_name tiene un  73.32 % de valores nulos
La columna title tiene un  75.02 % de valores nulos
La columna url tiene un  73.32 % de valores nulos
La columna release_date tiene un  75.04 % de valores nulos
La columna tags tiene un  73.46 % de valores nulos
La columna reviews_url tiene un  73.32 % de valores nulos
La columna specs tiene un  73.88 % de valores nulos
La columna price tiene un  74.46 % de valores nulos
La columna early_access tiene un  73.32 % de valores nulos
La columna id tiene un  73.32 % de valores nulos
La columna developer tiene un  76.06 % de valores nulos


In [35]:
#Eliminamos valores nulos, solo cuando toda la fila posee valores nulos:

steam_games = steam_games.dropna(how = 'all')

In [36]:
#Reindexamos el DataFrame:

steam_games = steam_games.reset_index(drop = True)

In [37]:
#Vemos las filas duplicadas:

cantidad_duplicados_games = steam_games.duplicated().sum()

cantidad_duplicados_games

0

In [38]:
#Corroboramos que no se visualice ninguna fila duplicada:

duplicados_games = steam_games[steam_games.duplicated()]

duplicados_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer


#### Filas duplicadas y valores nulos en 'reviews_desanidado':

In [39]:
user_reviews = reviews_desanidado

In [40]:
#Vemos los valores nulos por columna:

user_reviews.isna().sum()

funny          0
posted         0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
user_id        0
user_url       0
dtype: int64

In [41]:
#Corroboramos que no hay valores nulos, y por ende, el porcentaje de nulos por columna es de 0%:

f.porcentaje_valores_nulos(user_reviews)

La columna funny tiene un  0.00 % de valores nulos
La columna posted tiene un  0.00 % de valores nulos
La columna last_edited tiene un  0.00 % de valores nulos
La columna item_id tiene un  0.00 % de valores nulos
La columna helpful tiene un  0.00 % de valores nulos
La columna recommend tiene un  0.00 % de valores nulos
La columna review tiene un  0.00 % de valores nulos
La columna user_id tiene un  0.00 % de valores nulos
La columna user_url tiene un  0.00 % de valores nulos


In [42]:
#Vemos la cantidad de filas duplicadas:

cantidad_duplicados_reviews = user_reviews.duplicated().sum()

cantidad_duplicados_reviews

874

In [43]:
#Vemos cuales son las filas duplicadas:

duplicados_reviews = user_reviews[user_reviews.duplicated()]

duplicados_reviews

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
1112,,"Posted September 24, 2015.",,346110,1 of 1 people (100%) found this review helpful,True,yep,bokkkbokkk,http://steamcommunity.com/id/bokkkbokkk
2891,,"Posted January 10, 2014.",,218620,1 of 3 people (33%) found this review helpful,True,"Good graphics, fun heists! A bit laggy",ImSeriouss,http://steamcommunity.com/id/ImSeriouss
2892,,"Posted January 10, 2014.",,105600,0 of 2 people (0%) found this review helpful,True,So fun! DEFINITELY NOT RIP OFF OF MINECRAFT! e...,ImSeriouss,http://steamcommunity.com/id/ImSeriouss
2893,,"Posted December 17, 2014.",,570,No ratings yet,True,bobo pinoy,ImSeriouss,http://steamcommunity.com/id/ImSeriouss
2894,,"Posted January 13, 2014.",,211820,No ratings yet,True,If you want to play this game.. expect glithes...,ImSeriouss,http://steamcommunity.com/id/ImSeriouss
...,...,...,...,...,...,...,...,...,...
44433,,Posted July 3.,,422400,No ratings yet,True,Muy entretenido y una coleccion de armas prome...,76561198092022514,http://steamcommunity.com/profiles/76561198092...
44434,,Posted June 1.,,218620,No ratings yet,True,"Tiene una jugabilidad y tematica muy buena :D,...",76561198092022514,http://steamcommunity.com/profiles/76561198092...
44435,,"Posted August 17, 2014.",,261820,No ratings yet,True,"Buen juego, no importa el desarrrollo que tien...",76561198092022514,http://steamcommunity.com/profiles/76561198092...
44436,,"Posted February 17, 2014.",,224260,No ratings yet,True,exelente aporte :D¡¡¡ es una buen mod basado e...,76561198092022514,http://steamcommunity.com/profiles/76561198092...


In [44]:
#Eliminamos filas duplicadas, manteniendo la primer aparición:

user_reviews = user_reviews.drop_duplicates(keep = 'first')

In [45]:
#Verificamos que ya no hayan duplicados:

cantidad_duplicados_reviews = user_reviews.duplicated().sum()

cantidad_duplicados_reviews

0

In [46]:
#Reindexamos:

user_reviews = user_reviews.reset_index(drop = True)

#### Filas duplicadas en 'items_desanidado':

In [47]:
user_items = items_desanidado

In [48]:
#Vemos los valores nulos por columna:

user_items.isnull().sum()

item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
user_id             0
items_count         0
steam_id            0
user_url            0
dtype: int64

In [49]:
#Vemos el porcentaje de valores nulos por columa:

f.porcentaje_valores_nulos(user_items)

La columna item_id tiene un  0.00 % de valores nulos
La columna item_name tiene un  0.00 % de valores nulos
La columna playtime_forever tiene un  0.00 % de valores nulos
La columna playtime_2weeks tiene un  0.00 % de valores nulos
La columna user_id tiene un  0.00 % de valores nulos
La columna items_count tiene un  0.00 % de valores nulos
La columna steam_id tiene un  0.00 % de valores nulos
La columna user_url tiene un  0.00 % de valores nulos


In [50]:
#Vemos la cantidad de filas duplicadas:

cantidad_duplicados_items = user_items.duplicated().sum()

cantidad_duplicados_items

59104

In [51]:
#Vemos cuales son las filas duplicadas:

duplicados_items = user_items[user_items.duplicated()]

duplicados_items

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
164294,20,Team Fortress Classic,5,0,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad
164295,50,Half-Life: Opposing Force,0,0,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad
164296,70,Half-Life,0,0,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad
164297,130,Half-Life: Blue Shift,0,0,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad
164298,220,Half-Life 2,198,0,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad
...,...,...,...,...,...,...,...,...
4898223,213670,South Park™: The Stick of Truth™,725,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898224,221910,The Stanley Parable,53,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898225,261030,The Walking Dead: Season Two,253,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898226,273110,Counter-Strike Nexon: Zombies,0,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...


In [52]:
#Eliminamos los registros duplicados, manteniendo la primera aparición:

user_items = user_items.drop_duplicates(keep = 'first')

In [53]:
#Verificamos que ya no hayan registros duplicados:

cantidad_duplicados_items = user_items.duplicated().sum()

cantidad_duplicados_items

0

In [54]:
#Reindexamos:

user_items = user_items.reset_index(drop = True)

### Vemos otra vez, el tamaño de los DataFrames desanidados, sin nulos, y sin duplicados:

In [55]:
dataframes_limpios = {'steam_games': steam_games, 'user_reviews': user_reviews, 'user_items': user_items}

f.tamanio_datsets(dataframes_limpios)

El tamaño del DataFrame steam_games es: (32135, 13)
El tamaño del DataFrame user_reviews es: (58431, 9)
El tamaño del DataFrame user_items es: (5094105, 8)


### Modificación de algunos datos:

#### Modificaciones en 'steam_games':

In [56]:
#Vemos el tipo de dato de cada columna:

steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24064 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.2+ MB


Agregamos una columna con el año de lanzamiento:

In [57]:
#Vemos el formato de la fecha de lanzamiento:

anios = steam_games['release_date'].value_counts()

anios

release_date
2012-10-16    100
2017-08-31     92
2017-09-26     89
2017-06-21     82
2017-07-25     78
             ... 
1988-04-16      1
2013-08-24      1
2011-05-07      1
2010-08-21      1
2018-10-01      1
Name: count, Length: 3582, dtype: int64

In [58]:
#Agregamos una nueva columna, 'release_year', donde extrajimos el año de 'release_date':

steam_games['release_year'] = steam_games['release_date'].str.extract(r'(\b\d{4}\b)')

In [59]:
#Vemos no se pudieron extraer los años de 'release_date':

steam_games[['release_year', 'release_date']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   release_year  29964 non-null  object
 1   release_date  30068 non-null  object
dtypes: object(2)
memory usage: 502.2+ KB


In [60]:
#Vemos esos casos puntuales, donde 'release_year' es null, pero en 'release_date' son not null:

steam_games.loc[(steam_games.release_year.isna()) & (steam_games.release_date.notna())]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,release_year
10,Qucheza,"['Action', 'Indie', 'Simulation', 'Early Access']",Uncanny Islands,Uncanny Islands,http://store.steampowered.com/app/768570/Uncan...,Soon..,"['Early Access', 'Action', 'Indie', 'Simulatio...",http://steamcommunity.com/app/768570/reviews/?...,['Single-player'],NaN,1.0,768570.0,Qucheza,NaN
44,zha7,"['Adventure', 'Free to Play', 'RPG', 'Early Ac...",Idle Adventure,Idle Adventure,http://store.steampowered.com/app/714020/Idle_...,Beta测试已开启,"['Early Access', 'Free to Play', 'Adventure', ...",http://steamcommunity.com/app/714020/reviews/?...,"['MMO', 'Steam Achievements']",NaN,1.0,714020.0,"zha7,Phaedruns,Yingke Wu",NaN
6737,Ghost Machine,"['Action', 'Indie']",Warhawks,Warhawks,http://store.steampowered.com/app/380130/Warha...,Coming Soon,"['Action', 'Indie', 'VR', 'Flight']",http://steamcommunity.com/app/380130/reviews/?...,"['Single-player', 'Multi-player', 'Steam Achie...",NaN,0.0,380130.0,Ghost Machine,NaN
7134,Strange Loop Games,"['Adventure', 'Indie', 'Simulation']",Eco,Eco,http://store.steampowered.com/app/382310/Eco/,Coming Soon,"['Survival', 'Adventure', 'Multiplayer', 'Simu...",http://steamcommunity.com/app/382310/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",NaN,0.0,382310.0,Strange Loop Games,NaN
7189,"Alterego Games, Digital Forest","['Adventure', 'Casual', 'Indie']",Woven,Woven,http://store.steampowered.com/app/504690/Woven/,TBA,"['Indie', 'Casual', 'Adventure', 'Singleplayer...",http://steamcommunity.com/app/504690/reviews/?...,['Single-player'],NaN,0.0,504690.0,"Alterego Games,Digital Forest",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31983,8Floor,"['Casual', 'Strategy']",Lost Artifacts,Lost Artifacts,http://store.steampowered.com/app/708150/Lost_...,Coming soon,"['Strategy', 'Casual']",http://steamcommunity.com/app/708150/reviews/?...,"['Single-player', 'Steam Achievements']",NaN,0.0,708150.0,Creobit,NaN
31989,Dagestan Technology,"['Action', 'Adventure', 'Indie']",Memento,Memento,http://store.steampowered.com/app/454940/Memento/,TBD,"['Indie', 'Action', 'Adventure', 'Horror']",http://steamcommunity.com/app/454940/reviews/?...,['Single-player'],NaN,0.0,454940.0,CRAZY SECTOR,NaN
32000,Maulidan Games,"['Adventure', 'Indie']",Spy of Deimos,Spy of Deimos,http://store.steampowered.com/app/689820/Spy_o...,Coming Soon,"['Indie', 'Adventure']",http://steamcommunity.com/app/689820/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",NaN,0.0,689820.0,Maulidan Games,NaN
32008,New Blood Interactive,"['Action', 'Indie']",DUSK,DUSK,http://store.steampowered.com/app/519860/DUSK/,SOON™,"['Action', 'FPS', 'Indie', 'Retro', 'Shooter',...",http://steamcommunity.com/app/519860/reviews/?...,"['Single-player', 'Online Multi-Player', 'Stea...",20.0,0.0,519860.0,David Szymanski,NaN


In [61]:
#Hacemos Web Scrapping, para obtener 'release_date' de URL de la columna 'url':

steam_games['release_date_2'] = steam_games.loc[(steam_games.release_year.isna()) & (steam_games.release_date.notna()), 'url'].apply(f.obtener_ano_lanzamiento)

In [62]:
steam_games['release_date_2'].unique()

array([nan, '2 Mar, 2018', '13 Apr, 2018', '10 Jul, 2020', '6 Feb, 2018',
       '15 Nov, 2019', 'To be announced', '19 Jul, 2018', 'Coming soon',
       '22 Jan, 2018', '29 May, 2018', '13 Mar, 2018', None,
       '2 Mar, 2022', '24 Apr, 2018', '25 May, 2021', '28 Feb, 2018',
       '23 Feb, 2018', '13 Sep, 2022', '27 Mar, 2018', '31 Jan, 2019',
       '8 May, 2018', '7 Dec, 2021', '26 Jan, 2018', '18 Mar, 2019',
       '9 Apr, 2019', '23 May, 2020', '3 Dec, 2018', '20 Feb, 2019',
       '28 Aug, 2018', '15 Oct, 2019', '22 Feb, 2018', '20 Sep, 2018',
       '8 Mar, 2018', '31 Oct, 2019', '26 Nov, 2018', '11 Jul, 2018',
       '1 Jul, 2018', '4 Feb, 2019', '29 Oct, 2020', '27 Apr, 2022',
       '11 Jan, 2019', '12 Jul, 2018', '1 Oct, 2020', '5 Jul, 2019',
       '29 Mar, 2019', '19 Sep, 2019', '7 Mar, 2018', '5 Dec, 2018',
       'Aug 2018', '8 Jul, 2020', 'Q2 2024', '3 Aug, 2021',
       '30 Jul, 2018', '1 Feb, 2018', '10 Dec, 2018', '16 Mar, 2018'],
      dtype=object)

In [63]:
steam_games[['release_date', 'release_year', 'release_date_2']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   release_date    30068 non-null  object
 1   release_year    29964 non-null  object
 2   release_date_2  90 non-null     object
dtypes: object(3)
memory usage: 753.3+ KB


In [64]:
#Vemos por que algunas fechas de lanzamiento no se pudieron extraer

steam_games.loc[(steam_games.release_year.isna()) & (steam_games.release_date.notna()) & (steam_games.release_date_2).isna()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,release_year,release_date_2
11836,LEVER Software,"['Indie', 'RPG']",The Dungeon of Destiny,The Dungeon of Destiny,http://store.steampowered.com/app/555980/The_D...,TBD,"['RPG', 'Indie', 'Dungeon Crawler', 'Great Sou...",http://steamcommunity.com/app/555980/reviews/?...,"['Single-player', 'Steam Achievements', 'Capti...",NaN,0.0,555980.0,LEVER Software,NaN,None
12540,Siberian Digital,"['Adventure', 'Indie', 'RPG', 'Simulation', 'S...",Hard Era: The Fantasy Defence,Hard Era: The Fantasy Defence,http://store.steampowered.com/app/692260/Hard_...,TBD,"['Simulation', 'Strategy', 'Adventure', 'RPG',...",http://steamcommunity.com/app/692260/reviews/?...,['Single-player'],NaN,0.0,692260.0,Siberian Digital,NaN,None
12751,Dagestan Technology,"['Adventure', 'Casual', 'Indie', 'Strategy']",Fantasy World,Fantasy World,http://store.steampowered.com/app/708790/Fanta...,Soon,"['Adventure', 'Indie', 'Casual', 'Strategy']",http://steamcommunity.com/app/708790/reviews/?...,"['Single-player', 'Multi-player', 'Local Multi...",NaN,0.0,708790.0,LaserStone,NaN,None
12936,Enjoy Games,"['Adventure', 'Indie']",Lepur,Lepur,http://store.steampowered.com/app/767600/Lepur/,Soon,"['Adventure', 'Indie']",http://steamcommunity.com/app/767600/reviews/?...,['Single-player'],NaN,0.0,767600.0,"Sochnoslivka, Luis Zuno",NaN,None
13034,Dagestan Technology,"['Adventure', 'Casual', 'Indie']",Franky the Bumwalker,Franky the Bumwalker,http://store.steampowered.com/app/761310/Frank...,SOON,"['Casual', 'Adventure', 'Indie']",http://steamcommunity.com/app/761310/reviews/?...,['Single-player'],0.99,0.0,761310.0,Onlyjoy`s production,NaN,None
13103,Dagestan Technology,"['Action', 'Adventure', 'Indie']",Howl,Howl,http://store.steampowered.com/app/513640/Howl/,Soon,"['Adventure', 'Indie', 'Action']",http://steamcommunity.com/app/513640/reviews/?...,['Single-player'],NaN,0.0,513640.0,Gunsquad,NaN,None
13244,Dagestan Technology,"['Adventure', 'Casual', 'Indie']",Mad Quad,Mad Quad,http://store.steampowered.com/app/760450/Mad_Q...,SOON,"['Adventure', 'Indie', 'Casual']",http://steamcommunity.com/app/760450/reviews/?...,['Single-player'],NaN,0.0,760450.0,Retardia Games,NaN,None
13246,Dagestan Technology,"['Action', 'Adventure', 'Casual', 'Indie']",Dungeon Escapist,Dungeon Escapist,http://store.steampowered.com/app/760440/Dunge...,SOON,"['Action', 'Adventure', 'Indie', 'Casual']",http://steamcommunity.com/app/760440/reviews/?...,['Single-player'],NaN,0.0,760440.0,Retardia Games,NaN,None
13425,Atriagames,"['Action', 'Indie']",FYD,FYD,http://store.steampowered.com/app/760700/FYD/,Soon,"['Action', 'Indie']",http://steamcommunity.com/app/760700/reviews/?...,['Single-player'],NaN,0.0,760700.0,Atriagames,NaN,None
13614,u̷n̸k̴n̶o̴w̶n̵,"['Action', 'Adventure', 'Indie', 'Simulation',...",DONT PLAY THIS GAME.EXE,DONT PLAY THIS GAME.EXE,http://store.steampowered.com/app/679240/DONT_...,0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶...,"['Early Access', 'Violent', 'Action', 'Simulat...",http://steamcommunity.com/app/679240/reviews/?...,['Single-player'],NaN,1.0,679240.0,u̷n̸k̴n̶o̴w̶n̵,NaN,None


In [65]:
#Vemos más en detalle por que no se pudo extraer la fecha de lanzamiento de esas URLs.

steam_games.iloc[11836]

publisher                                            LEVER Software
genres                                             ['Indie', 'RPG']
app_name                                     The Dungeon of Destiny
title                                        The Dungeon of Destiny
url               http://store.steampowered.com/app/555980/The_D...
release_date                                                    TBD
tags              ['RPG', 'Indie', 'Dungeon Crawler', 'Great Sou...
reviews_url       http://steamcommunity.com/app/555980/reviews/?...
specs             ['Single-player', 'Steam Achievements', 'Capti...
price                                                           NaN
early_access                                                    0.0
id                                                         555980.0
developer                                            LEVER Software
release_year                                                    NaN
release_date_2                                  

Se pudo concluir que no se puede extraer la fecha de lanzamiento de esas URLs, ya que estas, nos dirigen directamente a la tienda principal de Steam, por ende, no va a ser posible extraerlas.

In [66]:
#Vemos el formato de las fechas que fueron extraidas con Web Scraping a traves de sus URLs.

steam_games['release_date_2'].unique()

array([nan, '2 Mar, 2018', '13 Apr, 2018', '10 Jul, 2020', '6 Feb, 2018',
       '15 Nov, 2019', 'To be announced', '19 Jul, 2018', 'Coming soon',
       '22 Jan, 2018', '29 May, 2018', '13 Mar, 2018', None,
       '2 Mar, 2022', '24 Apr, 2018', '25 May, 2021', '28 Feb, 2018',
       '23 Feb, 2018', '13 Sep, 2022', '27 Mar, 2018', '31 Jan, 2019',
       '8 May, 2018', '7 Dec, 2021', '26 Jan, 2018', '18 Mar, 2019',
       '9 Apr, 2019', '23 May, 2020', '3 Dec, 2018', '20 Feb, 2019',
       '28 Aug, 2018', '15 Oct, 2019', '22 Feb, 2018', '20 Sep, 2018',
       '8 Mar, 2018', '31 Oct, 2019', '26 Nov, 2018', '11 Jul, 2018',
       '1 Jul, 2018', '4 Feb, 2019', '29 Oct, 2020', '27 Apr, 2022',
       '11 Jan, 2019', '12 Jul, 2018', '1 Oct, 2020', '5 Jul, 2019',
       '29 Mar, 2019', '19 Sep, 2019', '7 Mar, 2018', '5 Dec, 2018',
       'Aug 2018', '8 Jul, 2020', 'Q2 2024', '3 Aug, 2021',
       '30 Jul, 2018', '1 Feb, 2018', '10 Dec, 2018', '16 Mar, 2018'],
      dtype=object)

In [67]:
#Extraemos nuevamente el año:

steam_games['release_year_2'] = steam_games['release_date_2'].str.extract(r'(\b\d{4}\b)')

In [68]:
#Verificamos:

steam_games['release_year_2'][7134]

'2018'

In [69]:
#Unimos la columna 'release_year' con la columna 'release_year_2'

steam_games['release_year'].fillna(steam_games['release_year_2'], inplace= True)

In [70]:
#Eliminamos las columnas 'release_date_2' y 'release_year_2'

steam_games = steam_games.drop(columns = ['release_year_2','release_date_2', 'release_date'])

Borramos valores nulos de la columna **release_year**:

In [71]:
steam_games['release_year'].unique()

array(['2018', '2017', nan, '1997', '1998', '2016', '2006', '2005',
       '2003', '2007', '2002', '2000', '1995', '1996', '1994', '2001',
       '1993', '2004', '1999', '2008', '2009', '1992', '1989', '2010',
       '2011', '2013', '2012', '2014', '1983', '1984', '2015', '1990',
       '1988', '1991', '1985', '1982', '1987', '2020', '2019', '1981',
       '1986', '2022', '2021', '1975', '1970', '1980', '2024'],
      dtype=object)

In [72]:
#Eliminamos las filas que contengan valores nulos en todas las columnas

steam_games = steam_games.dropna(how='all')

In [73]:
#Vemos el tamaño del DataFrame después de estas modificaciones

steam_games.shape

(32135, 13)

In [74]:
#Verificamos el porcentaje de valores nulos, para saber si eliminamos alguna columna, pero ninguna tiene un porcentaje mayor a 10%, asique las dejamos.

f.porcentaje_valores_nulos(steam_games)

La columna publisher tiene un  25.12 % de valores nulos
La columna genres tiene un  10.22 % de valores nulos
La columna app_name tiene un  0.01 % de valores nulos
La columna title tiene un  6.38 % de valores nulos
La columna url tiene un  0.00 % de valores nulos
La columna tags tiene un  0.51 % de valores nulos
La columna reviews_url tiene un  0.01 % de valores nulos
La columna specs tiene un  2.08 % de valores nulos
La columna price tiene un  4.29 % de valores nulos
La columna early_access tiene un  0.00 % de valores nulos
La columna id tiene un  0.01 % de valores nulos
La columna developer tiene un  10.27 % de valores nulos
La columna release_year tiene un  6.56 % de valores nulos


Verificamos duplicados según el id

In [75]:
steam_games.duplicated(subset = 'id').sum()

2

In [76]:
#Eliminamos duplicados según el id

steam_games = steam_games.drop_duplicates(subset = 'id')

In [77]:
#Verificamos:

steam_games.duplicated(subset = 'id').sum()

0

In [78]:
#Eliminamos la columna 'reviews_url', ya que es irrelevante.

steam_games = steam_games.drop(columns='reviews_url')

Descomponemos la columna 'genres', ya que tiene valores agrupados en listas:

In [79]:
# Primero, convertimos las cadenas en listas de Python usando ast.literal_eval.

steam_games['genres'] = steam_games['genres'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

# Luego, utilizamos el método apply con lambda para descomponer las listas.

steam_games = steam_games.drop('genres', axis=1).join(steam_games['genres'].apply(pd.Series).stack().reset_index(level=1, drop=True).rename('genres'))

C:\Users\ferna\AppData\Local\Temp\ipykernel_12180\211526033.py:7: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  steam_games = steam_games.drop('genres', axis=1).join(steam_games['genres'].apply(pd.Series).stack().reset_index(level=1, drop=True).rename('genres'))


In [80]:
#Verificamos que los valores no esten agrupados en listas:

steam_games['genres'].unique()

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', nan, 'Racing',
       'Early Access', 'Massively Multiplayer',
       'Animation &amp; Modeling', 'Video Production', 'Utilities',
       'Web Publishing', 'Education', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting'], dtype=object)

Hay un error al identificar '&', asique lo corregimos:

In [81]:
steam_games['genres'] = steam_games['genres'].str.replace('&amp;', 'and')

In [82]:
#Verificamos que no haya errores:

steam_games['genres'].unique()

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', nan, 'Racing',
       'Early Access', 'Massively Multiplayer', 'Animation and Modeling',
       'Video Production', 'Utilities', 'Web Publishing', 'Education',
       'Software Training', 'Design and Illustration', 'Audio Production',
       'Photo Editing', 'Accounting'], dtype=object)

In [83]:
steam_games

,publisher,app_name,title,url,tags,specs,price,early_access,id,developer,release_year,genres
0,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",['Single-player'],4.99,0.0,761140.0,Kotoshiro,2018,Action
0,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",['Single-player'],4.99,0.0,761140.0,Kotoshiro,2018,Casual
0,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",['Single-player'],4.99,0.0,761140.0,Kotoshiro,2018,Indie
0,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",['Single-player'],4.99,0.0,761140.0,Kotoshiro,2018,Simulation
0,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",['Single-player'],4.99,0.0,761140.0,Kotoshiro,2018,Strategy
...,...,...,...,...,...,...,...,...,...,...,...,...
32132,Laush Studio,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,"['Indie', 'Simulation', 'Racing']","['Single-player', 'Steam Achievements', 'Steam...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich,2018,Racing
32132,Laush Studio,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,"['Indie', 'Simulation', 'Racing']","['Single-player', 'Steam Achievements', 'Steam...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich,2018,Simulation
32133,SIXNAILS,EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...","['Single-player', 'Steam Achievements', 'Steam...",4.99,0.0,658870.0,"xropi,stev3ns",2017,Casual
32133,SIXNAILS,EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...","['Single-player', 'Steam Achievements', 'Steam...",4.99,0.0,658870.0,"xropi,stev3ns",2017,Indie


Analizamos la columna **price**:

In [84]:
steam_games['price'].unique()

array(['4.99', 'Free To Play', 'Free to Play', '0.99', '2.99', '3.99',
       '9.99', '18.99', '29.99', nan, 'Free', '10.99',
       '1.5899999999999999', '14.99', '1.99', '59.99', '8.99', '6.99',
       '7.99', '39.99', '19.99', '7.49', '12.99', '5.99', '2.49', '15.99',
       '1.25', '24.99', '17.99', '61.99', '3.49', '11.99', '13.99',
       'Free Demo', 'Play for Free!', '34.99', '74.76', '1.49', '32.99',
       '99.99', '14.95', '69.99', '16.99', '79.99', '49.99', '5.0',
       '44.99', '13.98', '29.96', '119.99', '109.99', '149.99', '771.71',
       'Install Now', '21.99', '89.99', 'Play WARMACHINE: Tactics Demo',
       '0.98', '139.92', '4.29', '64.99', 'Free Mod', '54.99', '74.99',
       'Install Theme', '0.89', 'Third-party', '0.5', 'Play Now',
       '299.99', '1.29', '3.0', '15.0', '5.49', '23.99', '49.0', '20.99',
       '10.93', '1.3900000000000001', 'Free HITMAN™ Holiday Pack',
       '36.99', '4.49', '2.0', '4.0', '9.0', '234.99',
       '1.9500000000000002', '1.5', '1

In [85]:
steam_games['price'] = steam_games['price'].fillna('--')

Vemos los valores que no son numericos y los guardamos en 'lista':

In [86]:
patron = r'^\D*$' #Patron para extraer los valores que no sean numericos

lista = steam_games[steam_games['price'].str.contains(patron, case=False, regex=True)]['price'].unique() #Extraemos los valores que no sean numericos

In [87]:
lista

array(['Free To Play', 'Free to Play', '--', 'Free', 'Free Demo',
       'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo',
       'Free Mod', 'Install Theme', 'Third-party', 'Play Now',
       'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Free to Try',
       'Free Movie', 'Free to Use'], dtype=object)

Vemos que los valores no numericos de la columna **price**, hacen referencia a que el juego es gratis. Entonces guardamos esos valores, exceptuando los nulos, en una lista vacia, recorriendolos con un blucle for:

In [88]:
lista_gratis = []

for i in lista:
    if i != '--':
        lista_gratis.append(i)
else:
    pass

Reemplazamos esos valores, por el valor '0.0':

In [89]:
steam_games['price'] = steam_games['price'].replace(lista, '0.0')

In [90]:
steam_games['price'].unique()

array(['4.99', '0.0', '0.99', '2.99', '3.99', '9.99', '18.99', '29.99',
       '10.99', '1.5899999999999999', '14.99', '1.99', '59.99', '8.99',
       '6.99', '7.99', '39.99', '19.99', '7.49', '12.99', '5.99', '2.49',
       '15.99', '1.25', '24.99', '17.99', '61.99', '3.49', '11.99',
       '13.99', '34.99', '74.76', '1.49', '32.99', '99.99', '14.95',
       '69.99', '16.99', '79.99', '49.99', '5.0', '44.99', '13.98',
       '29.96', '119.99', '109.99', '149.99', '771.71', '21.99', '89.99',
       '0.98', '139.92', '4.29', '64.99', '54.99', '74.99', '0.89', '0.5',
       '299.99', '1.29', '3.0', '15.0', '5.49', '23.99', '49.0', '20.99',
       '10.93', '1.3900000000000001', '36.99', '4.49', '2.0', '4.0',
       '9.0', '234.99', '1.9500000000000002', '1.5', '199.0', '189.0',
       '6.66', '27.99', '10.49', '129.99', '179.0', '26.99', '399.99',
       '31.99', '399.0', '20.0', '40.0', '3.33', '199.99', '22.99',
       '320.0', '38.85', '71.7', '59.95', '995.0', '27.49', '3.39', '6.0',


Hay valores con texto, pero que tambien tienen un precio, asique a esos, solo le dejaremos el precio:

In [91]:
steam_games['price'] = steam_games['price'].replace('Starting at $499.00', '499.00')

In [92]:
steam_games['price'] = steam_games['price'].replace('Starting at $449.00', '449.00')

In [93]:
steam_games['price'] = steam_games['price'].astype(float)

Guardamos el DataFrame 'steam_games', en un archivo csv:

In [94]:
steam_games.to_csv('../steam_games_limpio.csv', encoding= 'utf-8', index= False)

#### Modificaciones en 'user_reviews':

In [95]:
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58431 entries, 0 to 58430
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   funny        58431 non-null  object
 1   posted       58431 non-null  object
 2   last_edited  58431 non-null  object
 3   item_id      58431 non-null  object
 4   helpful      58431 non-null  object
 5   recommend    58431 non-null  bool  
 6   review       58431 non-null  object
 7   user_id      58431 non-null  object
 8   user_url     58431 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.6+ MB


In [96]:
#Vemos el formato de la fecha en la columna 'posted'

user_reviews['posted'][0]

'Posted November 5, 2011.'

Vemos el formato de la fecha en la columna 'posted', y es un formato incorrecto, ya que para nuestras funciones solo necesitamos el año:


In [97]:
#Lo primero que hacemos, es transformar la columna en tipo string, para extraer todo de cada valor, seguido de la palabra 'Posted':

user_reviews['posted'] = user_reviews['posted'].str.extract(r'(\b\d{4}\b)')

In [98]:
#Verificamos:

user_reviews['posted'].unique()

array(['2011', '2014', '2013', nan, '2015', '2012', '2010'], dtype=object)

In [99]:
user_reviews

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,,2011,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,2011,,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,,2011,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,,2014,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,,2013,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637
...,...,...,...,...,...,...,...,...,...
58426,,NaN,,70,No ratings yet,True,a must have classic from steam definitely wort...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
58427,,NaN,,362890,No ratings yet,True,this game is a perfect remake of the original ...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
58428,1 person found this review funny,NaN,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,LydiaMorley,http://steamcommunity.com/id/LydiaMorley
58429,,NaN,,730,No ratings yet,True,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley


In [100]:
user_reviews['posted'].unique()

array(['2011', '2014', '2013', nan, '2015', '2012', '2010'], dtype=object)

Guardamos el DataFrame 'user_reviews', en un archivo csv:

In [101]:
user_reviews.to_csv('../user_reviews_limpio.csv', encoding= 'utf-8', index= False)

El DataFrame 'user_items', no requiere de ninguna otra modificación, asique procedemos a guardarlo:

In [102]:
user_items.to_csv('../user_items_limpio.csv', encoding= 'utf-8', index= False)